In [ ]:
import Evaluation as eva

### SVM

In [ ]:
eva.evaluate("SVM","train_joined")

In [ ]:
eva.evaluate("SVM_missing_value","train_joined_missing_value")

In [ ]:
eva.evaluate("SVM_redundant","train_joined_redundant_value")

In [ ]:
eva.evaluate("SVM_undersample","train_joined_undersampled_value")

### LR

In [ ]:
eva.evaluate("LR","train_joined")

In [ ]:
eva.evaluate("LR_missing_value","train_joined_missing_value")

In [ ]:
eva.evaluate("LR_redundant","train_joined_redundant_value")

In [ ]:
eva.evaluate("LR_undersample","train_joined_undersampled_value")

### XGboost

In [ ]:
eva.evaluate("XGboost","train_joined")

In [ ]:
eva.evaluate("XGboost_missing_value","train_joined_missing_value")

In [ ]:
eva.evaluate("XGboost_redundant","train_joined_redundant_value")

In [ ]:
eva.evaluate("XGboost_undersample","train_joined_undersampled_value")

### Evaluate neural nets

In [ ]:
import torch
import numpy as np
import pandas as pd 
import sklearn

from torch import nn
from torch.utils.data import \
    Dataset as DS, \
    DataLoader as DL
from pandas import read_csv
from ipdb import set_trace

from AlexNet import AlexNet

import utils

In [ ]:
data_type = "standard"
X_train, X_test, Y_train, Y_test = utils.load_standard_data(data_type=data_type)

### testing configuration

In [ ]:
dropout = 0.5
lr = 0.0002
batch_size = 100
num_classes = 1
epochs = 100

In [ ]:
import metrics
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
_data_type_ = ""

try:
    del X_train
    del Y_train
except NameError:
    pass

input_X = torch.Tensor(X_test.values).unsqueeze(1)
AN = AlexNet(num_classes=num_classes, dropout=dropout)
# if torch.cuda.is_available():
#     input_X = input_X.cuda()
#     AN.cuda()


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix

AUCs = []
TNs, FPs, FNs, TPs = [], [], [], []
RECALLS = []

for i in range(5, 100, 5):
    _model_state_ = torch.load(f'{utils._data_pth_}/models/AN_{data_type}_epochs_{i}.pth', "cuda" if torch.cuda.is_available() else "cpu") 
    AN.load_state_dict(_model_state_)
    y_pred_probs = AN(input_X).detach().numpy()
    
    auc = roc_auc_score(Y_test, y_pred_probs)
    y_pred_probs[y_pred_probs >= 0.5] = 1
    y_pred_probs[y_pred_probs < 0.5] = 0
    tn, fp, fn, tp = confusion_matrix(Y_test, y_pred_probs).ravel()
    
    AUCs.append(auc)
    TNs.append(tn)
    FPs.append(tp)
    FNs.append(fn)
    TPs.append(fp)
    


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ax.plot(AUCs)
ax.plot(TNs)
ax.plot(np.array(TNs) / (np.array(TNs) + np.array(FNs)))
plt.show()

In [ ]:
from pandas import DataFrame

DataFrame({"AUC": AUCs, "TN": TNs, "FN": FNs, "TP": TPs}).to_csv(f"{utils._data_pth_}/models/AN_{data_type}_results.csv")